# DT Assignment1

# Data Loading

In [9]:
import pandas as pd 
import numpy as np
from itertools import combinations

In [10]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제

# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [11]:
def get_gini(df, label):
    Yes_num = 0
    No_num = 0
    for i in df[label]:
        if i == 'yes':
            Yes_num += 1
        else:
            No_num += 1

    gini = 1 - (((Yes_num / (Yes_num + No_num))**2 + (No_num / (Yes_num + No_num))**2))
    return gini

In [12]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [13]:
def get_binary_split(df, attribute):
    result = []
    temp = []

    kkk = set(list(combinations(df[attribute], 1))) #조합 중복제거하고 만들기

    for i in kkk:
        temp.append((list(i)[0])) # 리스트 형변환

    x = len(temp)
    for index in range(1 << x):
        result.append([temp[j] for j in range(x) if (index & (1 << j))]) # 부분조합 만들기

    return result[1:-1]

In [14]:
get_binary_split(pd_data, "age")

[['senior'],
 ['youth'],
 ['senior', 'youth'],
 ['middle_aged'],
 ['senior', 'middle_aged'],
 ['youth', 'middle_aged']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.
- 결과로 나온 Dictionary의 Key 값은 해당 class 들로 이루어진 tuple 형태로 들어가 있습니다.

In [15]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    feature = get_binary_split(df, attribute)

    for context in feature:
        context_num = 0 # 해당 속성 갯수
        not_context_num = 0 # 해당 속성 제외 갯수

        context_yes = 0 # 해당 속성 yes 갯수
        context_no = 0 # 해당 속성 no 갯수

        not_context_yes = 0 # 해당 속성 제외 yes 갯수
        not_context_no = 0 # 해당 속성 제외 no 갯수

        context_df = df[[attribute, label]] # 필요변수만 가지고 새로운 데이터프레임 생성

        for i in context_df.values:
            if i[0] in context and i[1] == 'yes':
                context_num += 1
                context_yes += 1

            elif i[0] in context and i[1] == 'no':
                context_num += 1
                context_no += 1

            elif i[0] not in context and i[1] == 'yes':
                not_context_num += 1
                not_context_yes += 1
            else:
                not_context_num += 1
                not_context_no += 1

        s = ",".join(context) # 딕셔너리에 넣을 키값 만들기, 리스트 형식을 문자열로

        # 아래는 수식계산입니다.
        result[s] = (context_num / (context_num + not_context_num)) * (1 - (
                (context_yes / (context_yes + context_no)) ** 2 + (context_no / (context_yes + context_no)) ** 2)) + \
                             (not_context_num / (context_num + not_context_num)) * (1 - (
                (not_context_yes / (not_context_yes + not_context_no)) ** 2 + (not_context_no / (not_context_yes + not_context_no)) ** 2))

    return result

In [16]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'senior': 0.45714285714285713,
 'youth': 0.3936507936507937,
 'senior,youth': 0.35714285714285715,
 'middle_aged': 0.35714285714285715,
 'senior,middle_aged': 0.3936507936507937,
 'youth,middle_aged': 0.45714285714285713}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

결과를 확인해보도록 하겠습니다.

In [17]:
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - senior,youth : 0.35714285714285715


# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [18]:
get_binary_split(pd_data, "income")

[['low'],
 ['medium'],
 ['low', 'medium'],
 ['high'],
 ['low', 'high'],
 ['medium', 'high']]

In [19]:
print(get_attribute_gini_index(pd_data, "age", "class_buys_computer"))
print(get_attribute_gini_index(pd_data, "income", "class_buys_computer"))
print(get_attribute_gini_index(pd_data, "student", "class_buys_computer"))
print(get_attribute_gini_index(pd_data, "credit_rating", "class_buys_computer"))

{'senior': 0.45714285714285713, 'youth': 0.3936507936507937, 'senior,youth': 0.35714285714285715, 'middle_aged': 0.35714285714285715, 'senior,middle_aged': 0.3936507936507937, 'youth,middle_aged': 0.45714285714285713}
{'low': 0.45, 'medium': 0.4583333333333333, 'low,medium': 0.4428571428571429, 'high': 0.4428571428571429, 'low,high': 0.4583333333333333, 'medium,high': 0.45}
{'yes': 0.3673469387755103, 'no': 0.3673469387755103}
{'excellent': 0.42857142857142855, 'fair': 0.42857142857142855}


여기서 age 변수의 지니변수가 다른변수와 비교하였을 때 가장 낮음

In [20]:
my_dict = get_attribute_gini_index(pd_data, "age", "class_buys_computer")
key_min = min(my_dict.keys(), key=(lambda k: my_dict[k]))
print('Min -',key_min, ":", my_dict[key_min])

Min - senior,youth : 0.35714285714285715
